In [1]:
# !pip install geemap

<h1><center>Monitoring Grassland Health in Mongolia </center></h1>

**Steps to create soum level change detection map:**

1. Choose Province
2. Choose Soum
3. Choose satellite derived index (more about these indices: https://www.usgs.gov/landsat-missions/landsat-surface-reflectance-derived-spectral-indices)
4. Select Year
5. Check `Only Pasture` (This will display areas that are deemed to be only grazing lands)
6. Check `Apply fmask(remove cloud, shadow, snow)` (This will use only images without any cloud, shadow)

**How to intrepret the map?**

The algorithm display vegetation change map for a given year for a given soum. The color maps can be intrepreted in relative to the mean of 8 years between 2014-2021. In other words, a color map indicates how much given year's vegetation deviate from from the mean value between 2014-2021. The three colors - green, yellow, red - represents the standard deviation distance. Green displays values where distance from mean is positive, yellow neatrul, and red negative. Thus, the intensity of the color indicates the larger distance from the mean.    

**Web Apps:** https://nogoonzun.herokuapp.com/

**Contact:** Khusel Avirmed (ta346@cornell.edu)

In [1]:
# <h1><center>Монгол Улсын Бэлчээрийн Мониторинг </center></h1>

# **Энэ сайтыг хэрхэн ашиглах вэ?**

# 1. Аймаг сонгох
# 2. Сум сонгох
# 3. Сансрын хиймэл дагуул ашиглан гаргасан мониторинг индех сонгох (Илүү дэлгэрэнгүй мэдээллийг: https://www.usgs.gov/landsat-missions/landsat-surface-reflectance-derived-spectral-indices)
# 4. Он сонгох
# 5. `Зөвхөн бэлчээр` зурган дээр гаргах 
# 6. `Үүл болон сүүдэрлэсэн хэсгийг хасах`

# **Газрын зурган дээр гарсан мэдээллийг хэрхэн тайлж унших вэ?**

# Доорх зурган мэдээллийг тухайн сумын сүүлийн 8 жилийн дундажтай харицуулж гаргасан болно. Өөрөөр хэлбэл, оны бэлчээрийн нөхцөл байдал сүүлийн 8 жилийн дундажаас хэр хазайсан нь ногоон, шар, болон улаан өнгөөр тайлагдаж уншигдана. Ногоон өнгө бэлчээр сайжирсан, шар өнгө өөрчлөлт бага, улаан өнгө бэлчээр доройтсонг харуулах бөгөөд, өнгөний тод байдал энэхүү өөрчлөлт хэр хүчтэй байгааг үзүүүлнэ.     

# **Web Apps:** https://nogoonzun.herokuapp.com/

# **Холбоо барих:** Khusel Avirmed (ta346@cornell.edu)

In [2]:
# Check geemap installation
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [3]:
# Import libraries
import os
import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl

In [4]:
# try:
#         ee.Initialize()
# except Exception as e:
#         ee.Authenticate()
#         ee.Initialize()

In [5]:
# Map.remove_ee_layer('Landsat 8' + ': ' + str(year_widget.value) + ' ' + str(nd_indices.value))

In [6]:
# Create an interactive map
Map = geemap.Map()
# Add Earth Engine data
fc = ee.FeatureCollection('users/ta346/mng-bounds/soum_aimag')

In [7]:
Map.add_basemap('ROAD')
Map.addLayer(ee.Image().paint(fc, 0, 2), {'palette': 'black'}, 'mng soums')

# states = ee.FeatureCollection('TIGER/2018/States')
# Map.addLayer(states, {}, 'US States')
Map.centerObject(fc, 5)
Map

Map(center=[46.96522531802441, 103.14224362261275], controls=(WidgetControl(options=['position', 'transparent_…

In [8]:
# bring soum province names as list of dictionaries
import json
with open("soum_province_names.json", "r") as read_file:
    newdict = json.load(read_file)

In [9]:
# Designe interactive widgets

style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='topleft')
Map.add_control(output_control)

# admin1_widget = widgets.Text(
#     description='State:',
#     value='Tennessee',
#     width=200,
#     style=style
# )

dc = 'Arkhangai'
province = widgets.Dropdown(
    options = list(newdict),
    description = 'Province:',
    disabled = False,
)

soum = widgets.Dropdown(
    options=newdict[dc],
    value = "Chuluut",
    description = 'Soum:',
    disabled = False,
)

def on_value_change(change):
    dc = change.new
    soum.options = newdict[dc]

province.observe(on_value_change, 'value')

nd_options = ['Normalized Difference Vegetation Index (NDVI)', 
              'Normalized Difference Water Index (NDWI)',
              'Modified Soil Adjusted Vegetation Index (MSAVI)',
              'Enhanced Vegetation Index (EVI)',
              'Near-Infrared Reflectence Vegetation (NIRv)']

nd_indices = widgets.Dropdown(options=nd_options, 
                              value='Normalized Difference Vegetation Index (NDVI)', 
                              description='Satellite Derived Index:', 
                              style=style)

year_widget = widgets.IntSlider(min=2014, max=2021, value=2014, description='Selected year:', width=400, style=style)

pasture_widget = widgets.Checkbox(
    value=True,
    description='Only pasture',
    style=style
)

fmask_widget = widgets.Checkbox(
    value=True,
    description='Apply fmask(remove cloud, shadow, snow)',
    style=style
)

submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

full_widget = widgets.VBox([
    widgets.VBox([province, soum, nd_indices, year_widget, pasture_widget, fmask_widget]),
    submit
])

full_widget

In [10]:
# states = fc.sort('aimag_eng', True)
# names = states.aggregate_array("aimag_eng").distinct().getInfo()

# newdict = {}
# for i in names:
#     dictionary = {
#         i : fc.filter(ee.Filter.eq("aimag_eng", i)).aggregate_array('soum_eng').distinct().getInfo()
#     }
    
#     newdict.update(dictionary)

# # write to json
# import json
# with open('soum_province_names.json', 'w') as fout:
#     json.dump(newdict , fout)

In [11]:
pasture = ee.Image("users/ta346/pasture_delineation/pas_raster_new")
# winter - 1, summer - 2, pasture-not-used - 3
pas = pasture # preserve all only pasture are. Pixels that don't have value of 0 in the datamask band (thos that are summer and winter pasture) get a value of 1
pas_winter = pasture.eq(1) # pixels that do have 1 in the image (those that are summer/fall pasture) will get value of 1, everything else 0
pas_summer = pasture.eq(2) # pixels that do have 2 in the image (those that are summer/fall pasture) will get value of 1, everything else 0

# define function to updateMask to apply to image collections
def mask_pas (img):
    return img.updateMask(pas).copyProperties(img, ['system:time_start'])
# define function to updateMask to apply to image collections
def mask_pas_winter (img):
    return img.updateMask(pas_winter).copyProperties(img, ['system:time_start'])
# define function to updateMask to apply to image collections
def mask_pas_summer (img):
    return img.updateMask(pas_summer).copyProperties(img, ['system:time_start'])

In [12]:
#ndvi function for image collections for landsat 8: C02 Coleection
def addNDVI_C2(im):
    ndvi = im.normalizedDifference(['SR_B4','SR_B3']).rename('ndvi').copyProperties(im, ['system:time_start'])
    return im.addBands(ndvi)

# soil adjusted vegetation index (SAVI) for landsat 8: C02 Coleection
def addSAVI_C2(image):
    savi = image.expression(
        '1.5 * (NIR - RED) / (NIR + RED + 0.5)', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3')}).rename('savi')
    return image.addBands(savi)

# ------Create an add MSAVI variable function for landsat 8: C02 Coleection
def addMSAVI_C2(image):
    msavi = image.expression(
        '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8*(NIR - RED)))/2', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3')}).rename('msavi')
    return image.addBands(msavi) #to find MSAVI2, divide by 2

# EVI for landsat 8: C02 Coleection
def addEVI_C2(im):
    evi = im.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': im.select('SR_B4'),
            'RED': im.select('SR_B3'),
            'BLUE': im.select('SR_B1')
            }).rename("evi")
    return im.addBands(evi)

# ------Create an add NIRv variable function for landsat 8: C02 Coleection
def addNIRv_C2(image):
    nirv = image.expression(
        'NIR * ((NIR - RED) / (NIR + RED))', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3')}).rename('nirv')
    return image.addBands(nirv)


# ------Create an add NDWI variable function for landsat 8: C02 Coleection
def addNDWI_C2(image):
    ndwi = image.expression(
        '(NIR - SWIR) / (NIR + SWIR)', {
            'NIR': image.select('SR_B4'),
            'SWIR': image.select('SR_B5')}).rename('ndwi')
    return image.addBands(ndwi)

# write function to download csv file from image collections
def yearlyComposite(startYear, endYear, shp):
    # function takes startYear, endYear, landcover mask (1985-1999: maskLC2000, 2000-2009: maskLC2010, 2010-2020: maskLC2020), shp = bag or soum, file name: str, folder_name: str (google drive folder to download to)
    startYear = startYear
    endYear = endYear
    stepList = ee.List.sequence(startYear, endYear)
    
    def func_qla(year):
        startDate = ee.Date.fromYMD(year,6,1)
        endDate = ee.Date.fromYMD(year,8,31)
        
        # Applies scaling factors.
        def applyScaleFactors(image):
            opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2).toFloat()
            thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0).toFloat()
            return image.addBands(opticalBands, overwrite = True) \
                      .addBands(thermalBands, overwrite = True)
        
        def func_ewa(img):
            dat = img.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7', 'QA_PIXEL'],
                             ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7', 'QA_PIXEL']).set('system:time_start', img.get('system:time_start'))
            
            qa = img.select('QA_PIXEL')

            # Bits 1 is diluted cloud; 3 cloud, and 5 cloud shadow
            dilatedCloud = (1 << 1)
            cloud = (1 << 3)
            cloudShadow = (1 << 4)

            # Both flags should be set to zero, indicating clear conditions.
            mask = qa.bitwiseAnd(dilatedCloud).eq(0) \
                        .And(qa.bitwiseAnd(cloud).eq(0)) \
                        .And(qa.bitwiseAnd(cloudShadow).eq(0))

            datMasked = dat.mask(mask)

            return datMasked
        
        ls8_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                                .filterBounds(shp) \
                                .filterDate(startDate, endDate) \
                                .map(applyScaleFactors) \
                                .map(func_ewa) 
        
        # apply indices functions and select only those bands
        ls8_collection_indices = ls8_collection.map(addNDVI_C2) \
                                            .map(addEVI_C2) \
                                            .map(addSAVI_C2) \
                                            .map(addMSAVI_C2) \
                                            .map(addNIRv_C2) \
                                            .map(addNDWI_C2) \
                                            .select('ndvi', 'evi', 'savi', 'msavi', 'nirv', 'ndwi')
        
        
        composite_i = ls8_collection_indices.median().clip(shp).set('system:time_start', startDate)
        
        return composite_i
    
    filterCollection = stepList.map(func_qla)

    yearlyComposites = ee.ImageCollection.fromImages(filterCollection) #return single image for each year
    
    return yearlyComposites

In [13]:
# Click event handler

def submit_clicked(b):
    
    try:
        Map.remove_ee_layer('Landsat 8' + ': ' + str(nd_indices.value))
    except Exception as e:
        print(e)
    
    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}
            
        try:
            province_id = province.value 
            soum_id = soum.value
            nd_indices_id = nd_indices.value
            selected_year = year_widget.value 
            pasture_yes = pasture_widget.value 
            apply_fmask = fmask_widget.value
                
            # select aimag and soum
            soum_aimag = fc.filter(ee.Filter.And(ee.Filter.eq("aimag_eng", province_id), ee.Filter.eq("soum_eng", soum_id)))
            layer_name = str(province_id) + ' ' + str(soum_id)
            geom = soum_aimag.geometry()
            
#             Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)
            
            if nd_indices_id == 'Normalized Difference Vegetation Index (NDVI)':
                ndviCollection = yearlyComposite(2014, 2021, geom).select('ndvi')
            elif nd_indices_id == 'Normalized Difference Water Index (NDWI)':
                ndviCollection = yearlyComposite(2014, 2021, geom).select('ndwi')
            elif nd_indices_id == 'Modified Soil Adjusted Vegetation Index (MSAVI)':
                ndviCollection = yearlyComposite(2014, 2021, geom).select('msavi')
            elif nd_indices_id == 'Enhanced Vegetation Index (EVI)':
                ndviCollection = yearlyComposite(2014, 2021, geom).select('evi')
            elif nd_indices_id == 'Near-Infrared Reflectence Vegetation (NIRv)':
                ndviCollection = yearlyComposite(2014, 2021, geom).select('nirv')
            
            if pasture_yes:
                ndviCollection = ndviCollection.map(mask_pas)
            else:
                ndviCollection = ndviCollection
            
            
            dateIni = ee.Date.fromYMD(selected_year, 1, 1)
            dateEnd = ee.Date.fromYMD(selected_year, 12, 31)
            
            selected_image = ndviCollection.filterDate(dateIni, dateEnd).first()
            

            yMean = ndviCollection.mean()
            stdImg = ndviCollection.reduce(ee.Reducer.stdDev())
            
            Anomaly = selected_image.subtract(yMean).divide(stdImg).clip(geom)
            
            # eviParams = {'min': 0, 'max': 1, 'palette': ['white', 'green']}

            colorizedVis = {
                'min': 0.0,
                'max': 1.0,
                'palette': [
                    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
                    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
                    '012E01', '011D01', '011301'
                ]}
            
            ndwiParams = {'min': -3, 'max': 3, 'palette': ['blue', 'deepskyblue', 'aqua']}
            
            anomParams = {'min': -3, 'max':3, 'palette': ['red', 'yellow', 'green']}
            
            anom_layer_name = 'Landsat 8' + ': ' + str(nd_indices_id)
            
            if nd_indices_id == 'Normalized Difference Water Index (NDWI)':
                Map.addLayer(Anomaly, ndwiParams, anom_layer_name)
            else:
                Map.addLayer(Anomaly, anomParams, anom_layer_name)
            
            Map.center_object(soum_aimag, 9)
            
            Map.default_style = {'cursor': 'default'}
            
#             Map.addLayer(selected_result_image, {'palette': palette}, 'Result ' + str(selected_year))

            
#             def cal_area(img):
#                 pixel_area = img.multiply(ee.Image.pixelArea())
#                 img_area = pixel_area.reduceRegion(**{
#                     'geometry': geom,
#                     'reducer': ee.Reducer.sum(),
#                     'scale': 1000,
#                     'maxPixels': 1e12,
#                     'bestEffort': True
#                 })
#                 return img.set({'area': img_area})
            
#             areas = result_images.map(cal_area)
#             stats = areas.aggregate_array('area').getInfo()
#             x = list(range(1984, 2021))
#             y = [item.get('nd') for item in stats]
            
#             fig = plt.figure(1)
#             fig.layout.height = '270px'
#             plt.clear()
#             plt.plot(x, y)
#             plt.title('Temporal trend (1984-2020)')
#             plt.xlabel('Year')
#             plt.ylabel('Area (ha)')
            
#             output_widget.clear_output()            

#             plt.show()
            
#             if download:
#                 out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#                 out_name = 'chart_' + geemap.random_string() + '.csv'
#                 out_csv = os.path.join(out_dir, out_name)
#                 if not os.path.exists(out_dir):
#                     os.makedirs(out_dir)
#                 with open(out_csv, 'w') as f:
#                     f.write('year, area (ha)\n')
#                     for index, item in enumerate(x):
#                         line = '{},{:.2f}\n'.format(item, y[index])
#                         f.write(line) 
#                 link = geemap.create_download_link(
#                     out_csv, title="Click here to download the chart data: ")
#                 display(link)
    
        except Exception as e:
            print(e)
            print('An error occurred during computation.')

submit.on_click(submit_clicked)